In [11]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import csv

In [2]:
df = pd.read_csv('main_task.csv')
df.loc[df['City'] == 'Oporto', 'City'] = 'Porto'
cities = df.City.unique()

In [4]:
df.head(2)

,Restaurant_id,City,Cuisine Style,Ranking,Rating,Price Range,Number of Reviews,Reviews,URL_TA,ID_TA
0,id_5569,Paris,"['European', 'French', 'International']",5570.0,3.5,$$ - $$$,194.0,"[['Good food at your doorstep', 'A good hotel ...",/Restaurant_Review-g187147-d1912643-Reviews-R_...,d1912643
1,id_1535,Stockholm,NaN,1537.0,4.0,NaN,10.0,"[['Unique cuisine', 'Delicious Nepalese food']...",/Restaurant_Review-g189852-d7992032-Reviews-Bu...,d7992032


In [323]:
import time
import requests
from bs4 import BeautifulSoup
import re

URL = df['URL_TA']
HEADERS = {'user-agent':
           'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.152 YaBrowser/21.2.2.101 Yowser/2.5 Safari/537.36',
           'accept': '*/*'}
rest_pars = []


def get_html(url, params=None):
    r = requests.get(url, headers=HEADERS, params=params)
    return r


def get_coord(url):
    r = requests.get(url, headers=HEADERS)
    soup = BeautifulSoup(r.text, 'html.parser')
    coord_finds = soup.find(content=re.compile(
        'https://maps.google.com/maps/api/'))
    if coord_finds:
        coord_finds = coord_finds.get('content').split(
            '&zoom')[0].split('?center=')[1].split('%2C')
    else:
        coord_finds = [[],[]]
    return coord_finds


def get_content(html):
    soup = BeautifulSoup(html, 'html.parser')
    pattern = re.compile('\d+')
    meals_find = soup.find('div', string='Meals')
    adress_find = soup.find(href="#MAPVIEW")
    rub_find = soup.find('div', string=re.compile('RUB'))

    if meals_find:
        meals_find = meals_find.find_next('div').get_text().split(', ')
    else:
        meals_find = 'no_info'

    if adress_find:
        adress_find = adress_find.get_text()
    else:
        adress_find = 'no_info'

    if rub_find:
        rub_find = pattern.findall(rub_find.get_text().replace(',', ''))
        rub_max = float(rub_find[1])
        rub_min = float(rub_find[0])
    else:
        rub_find = 'no_info'
        rub_max = 'no_info'
        rub_min = 'no_info'

    rest_pars.append({'Resraurant_id': df['Restaurant_id'][df['URL_TA'] == i].item(),
                      'meals': meals_find,
                      'rest_adress': adress_find,
                      'rest_coord': get_coord('https://google.com/maps/search/'+adress_find),
                      'rub_max': rub_max,
                      'rub_min': rub_min
                      })


def parse(url):
    html = get_html(url)
    if html.status_code == 200:
        get_content(html.text)
    else:
        pass


count = 1
col = URL[147]

for i in col:
    parse('https://www.tripadvisor.com' + i)
    print(round(count*100/len(col), 3), '%      ',
          count, '/', len(col), end='', flush=True)
    print('\r', end='')
    count += 1

ValueError: can only convert an array of size 1 to a Python scalar

In [319]:
rest_pars

[{'Resraurant_id': 'id_4429',
  'meals': ['BreakfastBrunchLunchDinner Breakfast  Brunch  Lunch  Dinner '],
  'rest_adress': 'no_info',
  'rest_coord': ['70.0732779', '24.26608125'],
  'rub_max': 'no_info',
  'rub_min': 'no_info'}]

In [312]:
data_rest = pd.DataFrame(rest_pars)
data_rest.rest_coord = data_rest.rest_coord.apply(lambda x: [float(x[0]), float(x[1])])
data_rest.to_csv('rest.csv', index = False)

In [313]:
data = pd.read_csv('rest.csv')
data

,Resraurant_id,meals,rest_adress,rest_coord,rub_max,rub_min
0,147 id_4110\n9446 id_4104\nName: Restau...,"['Breakfast', 'Lunch', 'Dinner']","Calle Espronceda 34, 28003 Madrid Spain","[40.4403981, -3.6949632]",no_info,no_info


In [308]:
print('https://www.tripadvisor.com' + URL[147])

https://www.tripadvisor.com/Restaurant_Review-g187514-d4600226-Reviews-Cafeteria_Teresa-Madrid.html


In [331]:
df.URL_TA.value_counts()[:20]

/Restaurant_Review-g187514-d9802625-Reviews-La_Vie_en_Doux-Madrid.html             2
/Restaurant_Review-g187514-d2477531-Reviews-Haya_19-Madrid.html                    2
/Restaurant_Review-g187514-d7337366-Reviews-Lady_Madonna-Madrid.html               2
/Restaurant_Review-g187514-d8753348-Reviews-Tropecientos-Madrid.html               2
/Restaurant_Review-g187514-d7342803-Reviews-Los_Hierros-Madrid.html                2
/Restaurant_Review-g187514-d9877640-Reviews-Raku_Sushi-Madrid.html                 2
/Restaurant_Review-g187514-d11921719-Reviews-Crepets-Madrid.html                   2
/Restaurant_Review-g187514-d10047799-Reviews-Entrepuntos-Madrid.html               2
/Restaurant_Review-g187514-d793170-Reviews-Caripen_Bistro-Madrid.html              2
/Restaurant_Review-g187514-d987900-Reviews-Compostela_Restaurante-Madrid.html      2
/Restaurant_Review-g187514-d11674463-Reviews-Beerhouse-Madrid.html                 2
/Restaurant_Review-g187514-d3161682-Reviews-Cerveceria_La_Surena-